
<h1 style='background-color:#D3D3D3; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;' > APTOS Blindness Detection with ResNet152V2 </h1>

<img src="https://www.researchgate.net/profile/Ankan-Ghosh-Dastider/publication/347866283/figure/fig2/AS:973637188321282@1609144603815/Architecture-of-the-proposed-classification-network-ResCovNet-ResNet152V2-has-been-used.ppm" width="800px">

* train.csv - the training labels
* train.zip - the training set images


저희는 해당 train data를 training, validation, test dataset (6:2:2) 으로 다시 분류하여 사용했습니다.

#### Dataset Link 

##### [Here](https://www.kaggle.com/c/aptos2019-blindness-detection/data)

In [ ]:
import os
import cv2
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from keras.models import Model
from keras import optimizers, applications
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

from typeguard import typechecked
from typing import Optional
%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

In [ ]:
train= pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
print('Number of train samples: ', train.shape[0]) # df.shape[0] : dataframe row count (행 개수 세기)

display(train.head()) # top 5 row (상위 5개 행 출력)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(train)

In [ ]:
f, ax = plt.subplots(figsize=(14, 5))
ax = sns.countplot(x="diagnosis", data=train, palette="Set2") # sns: seaborn, 카테고리별 데이터 개수 보여주는 그래프
sns.despine() # top, right 테두리 제거
plt.show() # 그래프 화면에 표시

# Down sampling
데이터의 불균형을 최소화 하기 위해,
data의 수가 많은 normal(0)과 moderate(2)의 데이터를 줄임

In [ ]:
normal = train[train.diagnosis == 0].sample(n=370, random_state=1004)
moderate = train[train.diagnosis == 2].sample(n=370, random_state=1004)
total = train[(train.diagnosis != 0) & (train.diagnosis != 2)].append(normal).append(moderate)

len(total)

In [ ]:
f, ax = plt.subplots(figsize=(14, 5))
ax = sns.countplot(x="diagnosis", data=total, palette="Set2") # sns: seaborn, 카테고리별 데이터 개수 보여주는 그래프
sns.despine() # top, right 테두리 제거
plt.show() # 그래프 화면에 표시

In [ ]:
sns.set_style("white") # 배경색 흰색으로 지정
count = 1
plt.figure(figsize=[15, 15]) # 새로운 figure 크기 가로 세로 15인치
for img_name in total['id_code'][:15]: # train의 id_code열 데이터 15행까지 받아옴
    img = cv2.imread("../input/preprocess-nb/preprocess/1/%s.png" % img_name)[...,[2, 1, 0]]
    plt.subplot(5, 5, count) # plt.subplot(nrow,ncol,pos) nrow : 몇 줄 ncol : 몇 칸 pos : 오른쪽, 아래 방향
    plt.imshow(img) # 그림 보여주기
    plt.title("Image %s" % count) # 제목 지정
    count += 1
    
plt.show()

In [ ]:
N_CLASSES = total['diagnosis'].nunique() # unique value 개수 나타냄
N_CLASSES

# Split data
train, test set으로 먼저 나눈다 (8:2)

In [ ]:
import numpy as np
x = total.iloc[:,0].values # total의 첫번째 열 값 추출
y = total.iloc[:,1].values
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=80)
# random_state로 seed 지정
# test, train split

In [ ]:
train = pd.DataFrame({'id_code':x_train,'diagnosis':y_train}) # 각 열 제목과 해당 값 넣어서 dataframe 만듦
test = pd.DataFrame({'id_code':x_test,'diagnosis':y_test})
train

In [ ]:
# Preprocecss data
train["id_code"] = train["id_code"].apply(lambda x: x + ".png") # train의 id_code 열 값에 .png 추가
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str') # train의 diagnosis 값 type을 모두 str로 변환
test['diagnosis'] = test['diagnosis'].astype('str')
print(train.head())
print(test.head())
print(len(train), len(test))

# model construction

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255, # pixel 값을 0~1 사이 값으로 변환
                                 validation_split=0.25, # validation data 비율 지정
                                 horizontal_flip=True) # 상하대칭

train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/preprocess-nb/preprocess/1/", # 해당 directory에서 가져옴
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16, # 한번에 네트워크에 넘겨주는 sample의 수
    class_mode="categorical",
    target_size=(224, 224),
    subset='training')

In [ ]:
valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/preprocess-nb/preprocess/1/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",    
    target_size=(224, 224),
    subset='validation')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory = "../input/preprocess-nb/preprocess/1/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=16,
        shuffle=False,
        class_mode=None)

In [ ]:
base_model = tf.keras.applications.ResNet152V2(input_shape=(224,224,3),include_top=False,weights="imagenet")

In [ ]:
# Freezing Layers

for layer in base_model.layers[:-10]:
    layer.trainable=False

In [ ]:
# Building Model

model=Sequential()
model.add(base_model)
# convolution, pooling을 통해 feature extraction
# layer 152개를 쌓아서 성능 높임

# 여기서부터 classifier
model.add(Dropout(0.5))
# overfitting 방지, 1-p의 확률로 특정 node를 제거하겠다.
model.add(Flatten())
# dense와 같이 분류를 위한 학습 레이어에서는 1차원 데이터 필요
# 1차원 데이터로 바꿔주는 역
model.add(BatchNormalization())
# gradient vanishing 방지 (layer 수 많을수록 심각)
model.add(Dense(256,kernel_initializer='he_uniform'))
# clssifier, 이미지를 적당한 카테고리로 분류
model.add(BatchNormalization())
model.add(Activation('relu'))
# 신경망의 output을 결정하는 식
model.add(Dropout(0.5))
model.add(Dense(128,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(5,activation='softmax')) # softmax 함수를 통해 최종 class 분류하기

# Model Summary


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png')

# f1_score 계산식

In [ ]:
def f1_score(y_true, y_pred): # taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# 모델 평가기준

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 2,verbose = 1,factor = 0.8, min_lr = 1e-6)
# learning rate를 조절해주는 콜백함수
# val_loss가 더이상 감소되지 않을 경우, 해당 함수 적용
# patience : 최적의 monitor 값을 기준으로 몇번의 epoch를 진행하고 leraning rate를 조절할지의 값
# factor : learning rate를 얼마나 감소시킬지 정하는 인자
# verbose : 1 = early stopping 적용될 때, 화면에 나타냄
# min_lr : learning rate의 하한선 지정
# 예를 들어 patience는 3이고, 30에폭에 정확도가 99%였을 때,만약 31번째에 정확도 98%, 32번째에 98.5%, 33번째에 98%라면 모델의 개선이 (patience=3)동안 개선이 없었기에,  ReduceLROnPlateau 콜백함수를 실행합니다.

mcp = ModelCheckpoint('ResNet152V2.h5')
# 모델을 저장할 때 사용되는 콜백함수

es = EarlyStopping(verbose=1, patience=2)
# 모델이 더이상 학습을 못할 경우(loss, metric 등의 개선이 없을 경우), 학습 도중 미리 학습을 종료시키는 콜백함수

## 참고 링크 : https://deep-deep-deep.tistory.com/55

In [ ]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=METRICS)
# optimizer : 손실함수 기반으로 어떻게 네트워크를 업데이트할 지 결정해줌
# loss : 손실 함수, 입력데이터와 출력데이터의 일치여부를 평가해주는 함수
# metrics : 평가 기준 (학습이 제대로 되고 있는지 살펴볼 수 있음)

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size +1
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size +1
# 1 epoch당 몇번 돌아야 하는지

In [ ]:
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

# model training

In [ ]:
%time
history = model.fit_generator(generator=train_generator,steps_per_epoch=STEP_SIZE_TRAIN,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,epochs=10,callbacks=[lrd,mcp,es])

In [ ]:
complete_datagen = ImageDataGenerator(rescale=1./255)
complete_generator = complete_datagen.flow_from_dataframe(  
        dataframe=train,
        directory = "../input/preprocess-nb/preprocess/1/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=1,
        shuffle=False,
        class_mode=None)

In [ ]:
STEP_SIZE_COMPLETE = complete_generator.n//complete_generator.batch_size
train_preds = model.predict_generator(complete_generator, steps=STEP_SIZE_COMPLETE)
train_preds = [np.argmax(pred) for pred in train_preds]

In [ ]:
len(train_preds)

# confusion matrix

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train['diagnosis'].astype('int'), train_preds)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
print(df_cm.describe().T)
plt.figure(figsize=(15, 8))
sns.heatmap(df_cm, annot=True, fmt='.2f')
plt.show()

In [ ]:
df_cm.head(5)

# training set에 대한 성능평가

In [ ]:
get_acc = history.history['accuracy']
value_acc = history.history['val_accuracy']
get_loss = history.history['loss']
validation_loss = history.history['val_loss']
get_auc = history.history['auc']
validation_auc = history.history['val_auc']
get_f1 = history.history['f1_score']
validation_f1 = history.history['val_f1_score']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_auc))
plt.plot(epochs, get_auc, 'r', label='auc of Training data')
plt.plot(epochs, validation_auc, 'b', label='auc of Validation data')
plt.title('Training vs validation auc')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_f1))
plt.plot(epochs, get_f1, 'r', label='f1 score of Training data')
plt.plot(epochs, validation_f1, 'b', label='f1 score of Validation data')
plt.title('Training vs validation f1 score')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
print("Train Cohen Kappa score: %.3f" % cohen_kappa_score(train_preds, train['diagnosis'].astype('int'), weights='quadratic'))

# test set prediction

In [ ]:
test_generator.reset()
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size +1
preds = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
predictions = [np.argmax(pred) for pred in preds]
predictions[:10]

In [ ]:
filenames = test_generator.filenames
results = pd.DataFrame(['id_code',filenames, 'diagnosis',predictions])
results.to_csv('submission.csv',index=False)
results.head(5).T

In [ ]:
print(len(filenames))
print(len(predictions))

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"id_code":filenames,
                      "diagnosis":predictions})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results.to_csv("submission.csv",index=False)

# test set의 실제 diagnosis value

In [ ]:
test.iloc[:,1].astype('int').hist()

# prediction value

In [ ]:
results.hist()

# test set confusion matrix

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(test['diagnosis'].astype('int'), predictions)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
print(df_cm.describe().T)
plt.figure(figsize=(15, 8))
sns.heatmap(df_cm, annot=True, fmt='.2f')
plt.show()

# test set에 대한 성능평가

In [ ]:
print("Test Cohen Kappa score: %.3f" % cohen_kappa_score(predictions, test['diagnosis'].astype('int'), weights='quadratic'))

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

list1=test['diagnosis'].astype('int').tolist()
acc=accuracy_score(list1, predictions)
rec=recall_score(list1, predictions,average='macro')
pre=precision_score(list1, predictions,average='macro')
f1=f1_score(list1, predictions, average='macro')
print(acc,rec,pre,f1)

In [ ]:
name = ['accuracy', 'recall', 'precision', 'f1_score']
score = [acc,rec,pre,f1]
df = pd.DataFrame({'evaluation':name,'score':score})

plt.figure(figsize=(8, 6))
splot=sns.barplot(x="evaluation",y="score",data=df)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.3f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=15,
                   xytext = (0, -12), 
                   textcoords = 'offset points')
plt.xlabel("evaluation", size=14)
plt.ylabel("score", size=14)
#plt.savefig("add_annotation_to_bars_in_barplot_Seaborn_Python.png")